# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from pprint import pprint
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [57]:
output_data = '/Users/yaden/Desktop/cities.csv'

cities_df = pd.read_csv(output_data)

cities_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [58]:
gmaps.configure(api_key=g_key)

In [59]:

degrees = cities_df[['Lat', 'Lng']]

humidity = cities_df['Humidity'].round(2)

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(degrees, humidity))

plt.savefig("humidity_heatmap") 

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [60]:

clean_df = cities_df.loc[(cities_df['Wind Speed'] <=10) & ((cities_df)['Cloudiness'] >=40) &
                        (cities_df['Max Temp'] >= 60) & (cities_df)["Max Temp"] <= 90 &
                        (cities_df['Humidity'] <=30)].dropna()

clean_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [72]:
hotel_df = clean_df.loc[:,['City', 'Country', 'Lat', 'Lng']]

hotel_df['Hotel Name'] = ''

hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,


In [89]:
# coordinates = ({hotel_df['Lat']},{hotel_df['Lng']})

hotel_names = []

# coordinates = f'{hotel_df['Lat']},{hotel_df['Lng']}'


params = {'type' : 'hotel', 
         'keyword' : 'hotel',
         'radius' : 5000, 
         'key' : g_key,
         'location' : (hotel_df['Lat'],hotel_df['Lng'])}

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

response = requests.get(base_url, params=params)

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row['Lng']
    
    params['location'] = f'{lat}, {lng}'
    
#     base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
#     params = hotel_df['Lat', 'Lng']
    
    results = requests.get(base_url, params=params)
    results = results.json()
    
    try:
        hotel_names.append(results['results'][0]['name'])
    except:
        hotel_names.append(np.nan)
        

pprint(hotel_names)

[nan,
 'Holiday Inn Mauritius Mon Tresor',
 'Qaanaaq Hotel',
 nan,
 'Atlanticview Cape Town Boutique Hotel',
 'Hôtel Hiva Oa Hanakee Lodge',
 'Eternity 141',
 'Umka',
 'Petra Tvoreniye',
 'Hotel Astoria',
 'Hotel boutique Palafito del Mar',
 nan,
 'Hotel Gobernador',
 'Renaissance Albany Hotel',
 'Gostinitsa "Mariya"',
 'Hotel Los Yámanas',
 'Laguna Beach Hotel & Spa',
 'Art-Otel\' " Kupecheskiy"',
 'Central Resort Ltd.',
 'The Royal Palms Resort',
 'Domaine de Rochevilaine',
 nan,
 'โรงแรมศิริชัย ณ กันตัง',
 'Anand Hotel',
 'Hotel Dreams Punta Arenas',
 "Pension Maro'i",
 'Tynwald Willow Bend Estate',
 'The Westin Qingdao',
 'Great Bear Lodge',
 nan,
 nan,
 'Long House Hotel Bethel',
 'Holiday Inn Express & Suites Clinton',
 'Huset ved havet',
 nan,
 nan,
 'Misty Waves Boutique Hotel',
 "Hampton Inn & Suites by Hilton St. John's Airport",
 'The Explorer Hotel',
 nan,
 'Finch Bay Galapagos Hotel',
 'Lokal A-Frame',
 nan,
 'Hotel Caleta Olivia',
 'Waldorf Astoria Los Cabos Pedregal',
 '

In [90]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [91]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))